In [209]:
from bs4 import BeautifulSoup
import requests

In [210]:
stout_url = 'https://www.danmurphys.com.au/dm/navigation/navigation_results.jsp?params=fh_location%3D%2F%2Fcatalog01%2Fen_AU%2Fcategories%3C%7Bcatalog01_2534374302025910%7D%2Fwebbeertype%3D%7Bdark20beer%7D%26fh_view_size%3D120%26fh_sort%3D-sales_value_30_days%26fh_modification%3D%2528secondid%253C%257Bdm_mystery57%257D%252Cblocked%2529%252C%2528secondid%253C%257Bdm_mystery52%257D%252Cblocked%2529%252C%2528secondid%253C%257Bdm_757646%257D%252Cblocked%2529%252C%2528secondid%253C%257Bdm_610528%257D%252Cblocked%2529%252C%2528secondid%253C%257Bdm_786481%257D%252Cblocked%2529%252C%2528secondid%253C%257Bdm_616722%257D%252Cblocked%2529%252C%2528secondid%253C%257Bdm_561685%257D%252Cblocked%2529%252C%2528instock%253D0%252Cbottom%2529&resetnav=false&storeExclusivePage=false'

In [211]:
headers = requests.utils.default_headers()

headers.update(
    {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
    }
)

In [212]:
result = requests.get(stout_url, headers=headers)
stout_page = result.content
with open('stouts.html','w') as f:
    to_write = stout_page.decode("utf-8", "ignore")
    f.write(to_write)

In [213]:
f = open('stouts.html')
soup = BeautifulSoup(f, 'html.parser')

In [214]:
products = soup.find_all("h3", { "class" : "title-product" }) 
len(products)

44

In [215]:
untapped_search_url = 'https://untappd.com/search'

In [216]:
def find_matching_beers(product_name):
    """
    Searches untappd with the given product name and returns results
    
    Args:
        product_name (string): The product name to search untappd for
        
    Returns:
        Array of matching html elements for untappd search results
    """
    result = requests.get(untapped_search_url, params={'q': product_name.replace('&','') })
    print(result.url)
    soup = BeautifulSoup(result.content, 'html.parser')
    matching_beers = soup.find_all('div', {'class': 'beer-item'})
    return matching_beers

def strip_stop_words(product_name, stop_words):
    """
    Strips out given stop words from product name
    
    Args:
        product_name (string): The product name to strip stop words from
        stop_words (list): The stop words to strip from the product name
    
    Returns:
        (string) the product name with stop words stripped from it
    """
    product_name_stripped = product_name
    for word in stop_words:
        product_name_stripped = product_name_stripped.replace(word, '')
    return product_name_stripped

In [224]:
for product in products:
    #product_name = product['alt'][:-6].replace('Cans','').replace('Bottles','')
    brand = product.span.text.strip()
    product_name_full = product.a['title']
    product_name = product_name_full[:-6].replace('Cans','').replace('Bottles','').strip()
    product_name_alone = product_name.replace(brand,'').strip()
    print(brand, product_name_alone)
    
    stop_words = ['&']
    matching_beers = find_matching_beers(strip_stop_words(product_name, stop_words))
    if(len(matching_beers) == 0):
        stop_words+=['Domestic Stout','&','and','The']
        matching_beers = find_matching_beers(strip_stop_words(product_name, stop_words))
        if(len(matching_beers) == 0):
            stop_words+=['The Black Box Of Dark Ales','Stout', 'Imperial','Ale','Polish', 
                         'Beer','Dark Lager','Russian','-', 'Doppelbock', 'Dark']
            polish = product_name.replace('Kozel Dark','Kozel Černý')
            matching_beers = find_matching_beers(strip_stop_words(polish, stop_words))
    
    for beer in matching_beers:
        name = beer.find('p', {'class', 'name'}).a.text.strip()
        rating = beer.find('span', {'class', 'num'}).text.replace('(','').replace(')','').strip()
        brewery = beer.find('p', {'class', 'brewery'}).a.text.strip()
        #print('?', brewery, name, rating)
        if (float(rating) > 0.1 
            and (
                brewery.split()[0] == brand.split()[0] 
                or name.split()[0] == brand.split()[0]
                or brand in ['Zywiec', 'Youngs']
                or brewery in ['BrewDog']
            )):
            print('*', brewery, name, rating)
    print()

Tooheys Old
https://untappd.com/search?q=Tooheys+Old
* Tooheys Brewing Old 3.138

White Rabbit Dark Ale
https://untappd.com/search?q=White+Rabbit+Dark+Ale
* White Rabbit Dark Ale 3.611
* White Rabbit Dark Ale Brewers Cut 3.845
* White Rabbit Kinda Dark Ale (Time Out Collab) 3.297
* White Rabbit Dark Christmas Pudding Ale 3.232

Monteith's Black Beer
https://untappd.com/search?q=Monteith%27s+Black+Beer
* Monteith's Brewing Co. Monteith's Black Beer 3.483

Coopers Dark Ale
https://untappd.com/search?q=Coopers+Dark+Ale
* Coopers Brewery Dark Ale 3.364
* Coopers Brewery Limited - Homebrews DIY Beer: Dark Ale 3.297

James Squire Jack of Spades Porter
https://untappd.com/search?q=James+Squire+Jack+of+Spades+Porter
* Malt Shovel Brewery James Squire Jack of Spades Porter 3.467

Kozel Dark Beer
https://untappd.com/search?q=Kozel+Dark+Beer
https://untappd.com/search?q=Kozel+Dark+Beer
https://untappd.com/search?q=Kozel+%C4%8Cern%C3%BD+
* Pivovar Velké Popovice Kozel Černý / Dark 3.335
* Pivovar 